Code for training neural network on LF data

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pickle

from data_handling_L1 import pre_process

np.random.seed(42)
### Import data from pickle
data_path = 'C:/Users/joarl/OneDrive/Dokument/Skola/Kand/Kod_git/kandidat/Low_freq_files/data_files_2s.pkl'

save_model_name = 'model_low_freq.pt'

In [2]:
# use GPU for computations if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open(data_path, 'rb') as f:
    data_list = pickle.load(f)

    
### prerocess ---------------------

data_processed = pre_process(data_list) 

labels = [t[1] for t in data_processed]
data = [t[2] for t in data_processed]

### split data (80/20) ------------------------------  

train_data = data[:11500]
train_labels = labels[:11500]

test_data = data[-3000:]
test_labels = labels[-3000:]

# create NumPy arrays from the data and labels
train_data = np.array(train_data)
test_data = np.array(test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

### convert into tensors ------------------------

train_data_reshaped = train_data.reshape(11500, 3, 512)
test_data_reshaped = test_data.reshape(3000, 3, 512)

train_labels_tensor = torch.from_numpy(train_labels).float()
train_data_tensor = torch.from_numpy(train_data_reshaped).float()
test_labels_tensor = torch.from_numpy(test_labels).float()
test_data_tensor = torch.from_numpy(test_data_reshaped).float()

#create teststet -----------------------------------------
trainset = TensorDataset(train_data_tensor, train_labels_tensor)
testset = TensorDataset(test_data_tensor, test_labels_tensor)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/joarl/OneDrive/Dokument/Skola/Kand/Kod_git/kandidat/Low_freq_files/data_files_2s.pkl'

In [4]:
#Define all parameters
total_iterations = 2000 # total number of iterations
batch_size = 100

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=128, kernel_size=8, stride=1)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, stride=1)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.tensor(x, dtype=self.conv1.weight.dtype).to(device)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.softmax(x)

        return x

In [5]:
def crossentropy(G, Y):
    # convert labels to onehot encoding
    Y_onehot = torch.eye(10)[Y]

    return -(Y_onehot * G.log()).sum(dim = 1).mean()

def accuracy(G, Y):
    return (G.argmax(dim=1) == Y).float().mean()


# initialize the test and training error statistics
test_accuracy = []
test_crossentropy = []
test_iter = []
train_accuracy = []
train_crossentropy = []
train_iter = []

# initialize the neural network and move it to the GPU if needed
net = ConvNet()
net.to(device)

# define the optimization algorithm
learningrate = 0.001  
optimizer = optim.Adam(net.parameters(), lr=learningrate)

# define the data loader for batches of the training data
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=1000, shuffle=True)

t = 0 # current iteration



In [6]:
done = False
while not done:
    for (batch_X, batch_Y) in trainloader:
        # move batch to the GPU if needed
        batch_Y = batch_Y.type(torch.LongTensor)
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        batch_G = net(batch_X)
        loss = F.cross_entropy(batch_G, batch_Y)
        # backpropagation
        loss.backward()
        # perform gradient descent step
        optimizer.step()
       
        with torch.no_grad():
            # evaluate the performance on the training data at every 5th iteration
            if t % 5 == 0:
                train_crossentropy.append(loss.item())
                train_accuracy.append(accuracy(batch_G, batch_Y).item())
                train_iter.append(t) 
                
            # evaluate the performance on the test data at every 100th iteration
            if t % 100 == 0:
                for (test_X, test_Y) in testloader:
                    # move batch to the GPU if needed
                    test_Y = test_Y.type(torch.LongTensor)
                    X, Y = test_X.to(device), test_Y.to(device)

                    # compute predictions for the test data
                    G = net(X)
                    test_crossentropy.append(F.cross_entropy(G, Y).item())
                    test_accuracy.append(accuracy(G, Y).item())
                    test_iter.append(t)

                    # print the iteration number and the accuracy of the predictions
                    print(f"Step {t:5d}: train accuracy {100 * train_accuracy[-1]:6.2f}% " \
                        f"train cross-entropy {train_crossentropy[-1]:5.2f}  " \
                        f"test accuracy {100 * test_accuracy[-1]:6.2f}% " \
                        f"test cross-entropy {test_crossentropy[-1]:5.2f}")
            
        # stop the training after the specified number of iterations
        t += 1
        if t > total_iterations:
            done = True
            break

torch.save(net.state_dict(), save_model_name)


C:\Users\joarl\AppData\Local\Temp\ipykernel_3688\2512127549.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=self.conv1.weight.dtype).to(device)


Step     0: train accuracy  80.00% train cross-entropy  0.67  test accuracy  97.60% test cross-entropy  0.46
Step     0: train accuracy  80.00% train cross-entropy  0.67  test accuracy  98.20% test cross-entropy  0.46
Step     0: train accuracy  80.00% train cross-entropy  0.67  test accuracy  97.20% test cross-entropy  0.46
Step   100: train accuracy  97.00% train cross-entropy  0.34  test accuracy  99.60% test cross-entropy  0.32
Step   100: train accuracy  97.00% train cross-entropy  0.34  test accuracy  98.70% test cross-entropy  0.33
Step   100: train accuracy  97.00% train cross-entropy  0.34  test accuracy  98.60% test cross-entropy  0.33
Step   200: train accuracy  98.00% train cross-entropy  0.33  test accuracy  99.30% test cross-entropy  0.32
Step   200: train accuracy  98.00% train cross-entropy  0.33  test accuracy  98.80% test cross-entropy  0.33
Step   200: train accuracy  98.00% train cross-entropy  0.33  test accuracy  99.20% test cross-entropy  0.32
Step   300: train a